### Moduļi

In [99]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm, logm, polar

### Konstantes

In [100]:
# Aproksimācijas precizitāte
EPSILON = 1e-3

# Izmantojamo vārtu kopa
GATESET = {}

tempqc = QuantumCircuit(1)
tempqc.rz(np.pi/4, 0)
GATESET[' Rz'] = tempqc
tempqc = tempqc.copy().inverse()
GATESET['-Rz'] = tempqc
del tempqc

tempqc = QuantumCircuit(1)
tempqc.h(0)
GATESET[' H '] = tempqc
del tempqc

# Izdrukā vārtu kopu
for gate_name in GATESET:
    print("Gate:", gate_name)
    print(Operator(GATESET[gate_name]).data)

# Pauli matricas 
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], complex)
Y = np.array([[0, -1j], [1j, 0]], complex)
Z = np.array([[1, 0], [0, -1]], complex)
PAULI_BASE = [X, Y, Z]

# Funkcija noņem globālo fāzi no unitāra operatora
def remove_global_phase(U):
    phase = np.angle(np.linalg.det(U)) / 2
    return U / np.exp(1j * phase)

Gate:  Rz
[[0.92387953-0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953+0.38268343j]]
Gate: -Rz
[[0.92387953+0.38268343j 0.        +0.j        ]
 [0.        +0.j         0.92387953-0.38268343j]]
Gate:  H 
[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]


### Saglabātās īsās ķēdes

In [101]:
# Īsas ķēdes ātrai piekļuvei aproksimācijai
SHORTHAND = []
SHORTHAND_LEN = 6

# Sāk ar īsajām ķēdēm ar vienu vārtu
for (gate_name, gate_qc) in GATESET.items():
    gate_U = Operator(gate_qc).data
    gate_U = remove_global_phase(gate_U)
    SHORTHAND.append((gate_name, gate_qc, gate_U))

# Visas iespējamās kombinācijas līdz SHORTHAND_LEN vārtiem
for (i) in range(1, SHORTHAND_LEN):
    for (name, qc, _) in SHORTHAND.copy():
        if (len(name.split()) == i):
            for (gate_name, gate_qc) in GATESET.items():
                new_name = name + ' ' + gate_name
                new_qc = qc.copy()
                new_qc.compose(gate_qc, [0], inplace=True)
                new_U = Operator(new_qc).data
                new_U = remove_global_phase(new_U)
                SHORTHAND.append((new_name, new_qc, new_U))

### Ievads

In [102]:
TARGET = np.sqrt(0.5) * np.array([
    [1, 1],
    [1j, -1j]
])
# TARGET = np.array([
#     [0, 1],
#     [1, 0]
# ])
# vai unitāra?
print("Vai unitāra?", np.allclose(TARGET @ TARGET.conj().T, I))
print("Aproksimējamais operators:")
print(TARGET)

Vai unitāra? True
Aproksimējamais operators:
[[0.70710678+0.j         0.70710678+0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]


In [103]:
# debug : izdrukā visas īsās ķēdes
for (name, _, U) in SHORTHAND:
    print(name, round(np.linalg.norm(TARGET - U, 2), 3))

 Rz 1.726
-Rz 1.465
 H  2.0
 Rz  Rz 1.848
 Rz -Rz 1.587
 Rz  H  1.999
-Rz  Rz 1.587
-Rz -Rz 1.414
-Rz  H  1.999
 H   Rz 1.111
 H  -Rz 1.962
 H   H  1.587
 Rz  Rz  Rz 1.935
 Rz  Rz -Rz 1.726
 Rz  Rz  H  1.983
 Rz -Rz  Rz 1.726
 Rz -Rz -Rz 1.465
 Rz -Rz  H  2.0
 Rz  H   Rz 1.986
 Rz  H  -Rz 1.955
 Rz  H   H  1.726
-Rz  Rz  Rz 1.726
-Rz  Rz -Rz 1.465
-Rz  Rz  H  2.0
-Rz -Rz  Rz 1.465
-Rz -Rz -Rz 1.465
-Rz -Rz  H  1.983
-Rz  H   Rz 1.986
-Rz  H  -Rz 1.955
-Rz  H   H  1.465
 H   Rz  Rz 1.848
 H   Rz -Rz 2.0
 H   Rz  H  1.726
 H  -Rz  Rz 2.0
 H  -Rz -Rz 1.848
 H  -Rz  H  1.465
 H   H   Rz 1.726
 H   H  -Rz 1.465
 H   H   H  2.0
 Rz  Rz  Rz  Rz 1.983
 Rz  Rz  Rz -Rz 1.848
 Rz  Rz  Rz  H  1.935
 Rz  Rz -Rz  Rz 1.848
 Rz  Rz -Rz -Rz 1.587
 Rz  Rz -Rz  H  1.999
 Rz  Rz  H   Rz 1.465
 Rz  Rz  H  -Rz 1.726
 Rz  Rz  H   H  1.848
 Rz -Rz  Rz  Rz 1.848
 Rz -Rz  Rz -Rz 1.587
 Rz -Rz  Rz  H  1.999
 Rz -Rz -Rz  Rz 1.587
 Rz -Rz -Rz -Rz 1.414
 Rz -Rz -Rz  H  1.999
 Rz -Rz  H   Rz 1.111
 Rz -Rz  H  -Rz 1.

### Funkcijas

In [104]:
# Rekursīvais algoritms, kas balstās uz Soloveja-Kitājeva teorēmas
def solovay_kitaev_decomposition(U_target, depth=3):
    
    U_target = remove_global_phase(U_target)

    if (depth == 0):
        return shorthand_approximation(U_target)
    
    qc_approx = solovay_kitaev_decomposition(U_target, depth-1)
    U_approx = Operator(qc_approx).data

    A, B = gc_decomposition(U_target @ U_approx.conj().T)

    qc_A = solovay_kitaev_decomposition(A, depth-1)
    qc_B = solovay_kitaev_decomposition(B, depth-1)
    qc_A_inv = qc_A.inverse()
    qc_B_inv = qc_B.inverse()

    # Izveido jaunu ķēdi
    qc = QuantumCircuit(1)
    qc.compose(qc_A, [0], inplace=True)
    qc.compose(qc_B, [0], inplace=True)
    qc.compose(qc_A_inv, [0], inplace=True)
    qc.compose(qc_B_inv, [0], inplace=True)
    qc.compose(qc_approx, [0], inplace=True)

    return qc

# Funkcija, kas atrod labāko īso ķēdi dotajam operatoram
# izmanto rekursijas bāzes gadījumā
def shorthand_approximation(U_target):
    print("Ātrais tuvinājums operatoram:")
    print(U_target)

    min_error = float('inf')
    best_entry = SHORTHAND[0]
    for entry in SHORTHAND:
        error = np.linalg.norm(U_target - entry[2], 2)
        if error < min_error:
            min_error = error
            best_entry = entry
    print(f"Tuvākā ķēde {best_entry[0]} ar kļūdu {min_error}")

    return best_entry[1] # atgriež QuantumCircuit

# Grupas komutatora dekompozīcija
def gc_decomposition(U):
    print("---GC DEKOMPOZĪCIJA---")
    print("Grupas komutatora dekompozīcija operatoram:")
    print(U)

    U = remove_global_phase(U)
    print("Bez globālās fāzes:")
    print(U)

    # Izvelk rotācijas asi un leņķi
    axis, phi = extract_axis_angle(U)

    print("angle", phi)
    
    # Sadala rotācijās ar ortonormālām asīm un vienādiem leņķiem
    theta = np.sqrt(phi)

    w = [0, 0, 1] # sāk ar Z asi
    if abs(np.dot(axis, w)) > 0.9:
        w = [0, 1, 0] # ja tuvu, izvēlas Y asi

    # Veido ortonormālo bāzi
    w = np.array(w)
    u = w - np.dot(w, axis) * axis
    u = u / np.linalg.norm(u)
    v = np.cross(axis, u)
    print(np.dot(axis, u))
    print("Ortonormēta bāze:")
    print("u:", u)
    print("v:", v)

    # Rotācijas matricas ap asīm u un v
    A = rotation_matrix(u, theta)
    B = rotation_matrix(v, theta)

    # Tests
    U_reconstructed = A @ B @ A.conj().T @ B.conj().T
    print("Atjaunotais operators no dekompozīcijas:")
    print(U_reconstructed)
    print("Atšķirība starp sākotnējo un atjaunoto operatoru:")
    print(np.linalg.norm(U - U_reconstructed, 2))
    print("---")
    
    return A, B

# Funkcija izvelk rotācijas asi un leņķi no unitāra operatora
def extract_axis_angle(R):

    # Aprēķina leņķi
    trace = np.trace(R)
    theta = np.arccos(np.real(trace) / 2) * 2

    # Ja leņķis ir tuvu nullei, atgriež standarta asi un nulles leņķi
    if np.isclose(theta, 0, atol=1e-12):
        return np.array([1, 0, 0]), 0

    # Aprēķina rotācijas asi
    A = (R - np.cos(theta / 2) * I) / (-1j * np.sin(theta / 2))
    nx = np.real(A[1, 0])
    ny = np.imag(A[1, 0])
    nz = np.real(A[0, 0])
    axis = np.array([nx, ny, nz]) / np.linalg.norm([nx, ny, nz])

    return axis, theta

def rotation_matrix(axis, theta):
    return expm(-1j * theta / 2 * (axis[0] * X + axis[1] * Y + axis[2] * Z))

### Main bloks

In [105]:
final_circuit = solovay_kitaev_decomposition(TARGET, 2)

final_operator = Operator(final_circuit).data
print("Final circuit operator:")
print(final_operator)

print("Final approximation error:", np.linalg.norm(TARGET - final_operator, 2))

final_circuit.draw()

Ātrais tuvinājums operatoram:
[[ 0.5+0.5j  0.5+0.5j]
 [-0.5+0.5j  0.5-0.5j]]
Tuvākā ķēde -Rz -Rz  H  -Rz -Rz  H  ar kļūdu 3.04047097224406e-16
---GC DEKOMPOZĪCIJA---
Grupas komutatora dekompozīcija operatoram:
[[ 1.00000000e+00+5.55111512e-17j -1.11022302e-16+0.00000000e+00j]
 [ 1.11022302e-16+0.00000000e+00j  1.00000000e+00-5.55111512e-17j]]
Bez globālās fāzes:
[[ 1.00000000e+00+5.55111512e-17j -1.11022302e-16+0.00000000e+00j]
 [ 1.11022302e-16+0.00000000e+00j  1.00000000e+00-5.55111512e-17j]]
angle 4.2146848510894035e-08
-5.551115123125783e-17
Ortonormēta bāze:
u: [0.         0.4472136  0.89442719]
v: [ 1.  0. -0.]
Atjaunotais operators no dekompozīcijas:
[[ 1.00000000e+00+9.42238691e-09j -1.88496108e-08+2.16315515e-12j]
 [ 1.88496108e-08+2.16315509e-12j  1.00000000e+00-9.42238691e-09j]]
Atšķirība starp sākotnējo un atjaunoto operatoru:
2.1073424106945843e-08
---
Ātrais tuvinājums operatoram:
[[ 9.99999995e-01-9.18115988e-05j -4.59057994e-05+4.13590306e-25j]
 [ 4.59057994e-05+0.00000

┌───┐┌───┐┌───┐┌─────────┐┌──────────┐┌───┐┌───┐┌───┐┌───┐┌─────────┐»
q: ┤ H ├┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ H ├┤ H ├┤ H ├┤ Rz(π/4) ├»
   └───┘└───┘└───┘└─────────┘└──────────┘└───┘└───┘└───┘└───┘└─────────┘»
«   ┌──────────┐┌───┐┌─────────┐┌─────────┐┌──────────┐┌───┐┌───┐┌──────────┐»
«q: ┤ Rz(-π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ H ├┤ Rz(-π/4) ├»
«   └──────────┘└───┘└─────────┘└─────────┘└──────────┘└───┘└───┘└──────────┘»
«   ┌─────────┐┌─────────┐┌──────────┐┌─────────┐┌──────────┐┌──────────┐┌───┐»
«q: ┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ H ├»
«   └─────────┘└─────────┘└──────────┘└─────────┘└──────────┘└──────────┘└───┘»
«   ┌──────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐┌─────────┐┌─────────┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├»
«   └──────────┘└─────────┘└───┘└──────────┘└─────────┘└─────────┘└─────────┘»
«   ┌──────────┐┌─────────┐┌──────────┐┌──────────┐┌──────────┐┌─────────┐┌───┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├»
«   └──────────┘└─────────┘└──────────┘└──────────┘└──────────┘└─────────┘└───┘»
«   ┌──────────┐┌─────────┐┌───┐┌───┐┌─────────┐┌──────────┐┌───┐┌─────────┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ Rz(π/4) ├»
«   └──────────┘└─────────┘└───┘└───┘└─────────┘└──────────┘└───┘└─────────┘»
«   ┌──────────┐┌─────────┐┌───┐┌───┐┌─────────┐┌──────────┐┌──────────┐┌───┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ H ├»
«   └──────────┘└─────────┘└───┘└───┘└─────────┘└──────────┘└──────────┘└───┘»
«   ┌─────────┐┌──────────┐┌───┐┌───┐┌───┐┌───┐┌─────────┐┌──────────┐┌───┐»
«q: ┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ H ├┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├»
«   └─────────┘└──────────┘└───┘└───┘└───┘└───┘└─────────┘└──────────┘└───┘»
«   ┌───┐┌───┐┌─────────┐┌──────────┐┌───┐┌─────────┐┌──────────┐┌───┐┌───┐»
«q: ┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ H ├»
«   └───┘└───┘└─────────┘└──────────┘└───┘└─────────┘└──────────┘└───┘└───┘»
«   ┌──────────┐┌─────────┐┌───┐┌──────────┐┌─────────┐┌─────────┐┌─────────┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├»
«   └──────────┘└─────────┘└───┘└──────────┘└─────────┘└─────────┘└─────────┘»
«   ┌──────────┐┌─────────┐┌──────────┐┌──────────┐┌──────────┐┌─────────┐┌───┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├»
«   └──────────┘└─────────┘└──────────┘└──────────┘└──────────┘└─────────┘└───┘»
«   ┌──────────┐┌─────────┐┌───┐┌─────────┐┌─────────┐┌──────────┐┌─────────┐»
«q: ┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├»
«   └──────────┘└─────────┘└───┘└─────────┘└─────────┘└──────────┘└─────────┘»
«   ┌──────────┐┌──────────┐┌─────────┐┌─────────┐┌──────────┐┌───┐┌───┐»
«q: ┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ H ├┤ H ├»
«   └──────────┘└──────────┘└─────────┘└─────────┘└──────────┘└───┘└───┘»
«   ┌──────────┐┌───┐┌──────────┐┌─────────┐┌───┐┌─────────┐┌──────────┐»
«q: ┤ Rz(-π/4) ├┤ H ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├»
«   └──────────┘└───┘└──────────┘└─────────┘└───┘└─────────┘└──────────┘»
«   ┌─────────┐┌───┐┌───┐┌─────────┐┌──────────┐┌──────────┐┌─────────┐»
«q: ┤ Rz(π/4) ├┤ H ├┤ H ├┤ Rz(π/4) ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ Rz(π/4) ├»
«   └─────────┘└───┘└───┘└─────────┘└──────────┘└──────────┘└─────────┘»
«   ┌──────────┐┌───┐┌──────────┐┌─────────┐┌───┐┌──────────┐┌──────────┐┌───┐»
«q: ┤ Rz(-π/4) ├┤ H ├┤ Rz(-π/4) ├┤ Rz(π/4) ├┤ H ├┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ H ├»
«   └──────────┘└───┘└──────────┘└─────────┘└───┘└──────────┘└──────────┘└───┘»
«   ┌──────────┐┌──────────┐┌───┐
«q: ┤ Rz(-π/4) ├┤ Rz(-π/4) ├┤ H ├
«   └──────────┘└──────────┘└───┘